<a href="https://colab.research.google.com/github/hanmrf/MachineLearning/blob/main/apple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1

In [ ]:
# Impor library yang dibutuhkan
from IPython import get_ipython
from IPython.display import display
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from google.colab import files
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image, ImageDraw, ImageOps # Impor Image dan ImageDraw dari PIL

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download muhammadraihanf12/agrocurify-apple

Dataset URL: https://www.kaggle.com/datasets/muhammadraihanf12/agrocurify-apple
License(s): unknown
 98% 83.0M/84.3M [00:05<00:00, 22.5MB/s]
100% 84.3M/84.3M [00:05<00:00, 17.1MB/s]


In [ ]:
import zipfile

# Lokasi file ZIP
zip_file_path = "/content/agrocurify-apple.zip"
extract_to_path = "/content/"

# Mengekstrak file ZIP
if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)
        print("File berhasil diekstrak ke direktori:", extract_to_path)
else:
    print("File ZIP tidak ditemukan di lokasi:", zip_file_path)

File berhasil diekstrak ke direktori: /content/


In [ ]:
apple_data = "/content/apple"
apple_classes = os.listdir(apple_data)

print("apple_classes:")
for classes in apple_classes:
    if os.path.isdir(os.path.join(apple_data, classes)):
        print(classes)

apple_classes:
Not___apple
Apple___Apple_scab
Apple___Cedar_apple_rust
Apple___Black_rot
Apple___healthy


In [ ]:
for item in apple_classes:
    print("\nClass Name:", item)
    class_dir = os.path.join(apple_data, item)
    class_images = os.listdir(class_dir)

    # Total images in each class
    num_images = len(class_images)
    print("Total Images:", num_images)

    # Resolution/size of the first image in the class
    img_path = os.path.join(class_dir, class_images[0])
    img = mpimg.imread(img_path)
    image_shape = img.shape
    print("Resolution of First Image:", image_shape)

    # Displaying some images with descriptive labels
    plt.figure(figsize=(15, 5))
    plt.suptitle(f"Class: {item} (Total: {num_images} Images)", fontsize=16)

    for i, img_file in enumerate(class_images[:5]):  # Display up to 5 images
        img_path = os.path.join(class_dir, img_file)
        img = mpimg.imread(img_path)

        # Add a subplot for each image
        sp = plt.subplot(1, 5, i + 1)
        plt.axis('off')
        plt.imshow(img)

        # Add descriptive title to each image
        plt.title(f"Img {i+1}\n{img.shape[0]}x{img.shape[1]} px", fontsize=10)

    plt.tight_layout()
    plt.subplots_adjust(top=0.85)  # Adjust to fit the suptitle
    plt.show()

# Preprocesing Dataset

In [ ]:
# New Directory for Resized Dataset
resized_dir = "/content/resized"
os.makedirs(resized_dir, exist_ok=True)

# Pixel Normalization
def normalize_image(image):
    normalized_image = image.astype(np.float32) / 255.0
    return normalized_image

# Fungsi resize_with_aspect_ratio
def resize_with_aspect_ratio(image, target_size):
    image.thumbnail(target_size, Image.Resampling.LANCZOS)  # Ganti dengan Image.Resampling.LANCZOS

    # Membuat gambar baru dengan ukuran target dan menempelkan gambar yang sudah diubah ukurannya
    new_image = Image.new("RGB", target_size, (255, 255, 255))
    new_image.paste(image, ((target_size[0] - image.size[0]) // 2, (target_size[1] - image.size[1]) // 2))

    return new_image

    # Membuat gambar baru dengan ukuran target
    new_image = Image.new("RGB", target_size, (255, 255, 255))  # Gambar putih sebagai latar belakang
    new_image.paste(image, ((target_size[0] - image.size[0]) // 2, (target_size[1] - image.size[1]) // 2))
    return new_image

for item in apple_classes:
    class_dir = os.path.join(apple_data, item)
    class_images = os.listdir(class_dir)

    for img_path in class_images:
        img = Image.open(os.path.join(class_dir, img_path))
        resized_img = resize_with_aspect_ratio(img, (224, 224))
        normalized_img = normalize_image(np.array(resized_img))

        # Save Image to resized_dir
        save_path = os.path.join(resized_dir, item, img_path)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        Image.fromarray((normalized_img * 255).astype(np.uint8)).save(save_path)

In [ ]:
# New Directory for Balanced Dataset
balanced_dir = "/content/balanced"

# Number of Images to Be Selected from Each Class
num_images_per_class = min(len(os.listdir(resized_dir+"/Apple___Apple_scab")),
                           len(os.listdir(resized_dir+"/Apple___Black_rot")),
                           len(os.listdir(resized_dir+"/Apple___Cedar_apple_rust")),
                           len(os.listdir(resized_dir+"/Apple___healthy")),
                           len(os.listdir(resized_dir+"/Not___apple")))

# Create New Directory in balanced_dir
os.makedirs(balanced_dir+"/Apple___Apple_scab", exist_ok=True)
os.makedirs(balanced_dir+"/Apple___Black_rot", exist_ok=True)
os.makedirs(balanced_dir+"/Apple___Cedar_apple_rust", exist_ok=True)
os.makedirs(balanced_dir+"/Apple___healthy", exist_ok=True)
os.makedirs(balanced_dir+"/Not___apple", exist_ok=True)

# Select Some Images from Each Class and Copy to balanced_dir
selected_images = os.listdir(resized_dir+"/Apple___Apple_scab")[:num_images_per_class]
for image in selected_images:
    shutil.copy(os.path.join(resized_dir+"/Apple___Apple_scab", image), balanced_dir+"/Apple___Apple_scab")

selected_images = os.listdir(resized_dir+"/Apple___Black_rot")[:num_images_per_class]
for image in selected_images:
    shutil.copy(os.path.join(resized_dir+"/Apple___Black_rot", image), balanced_dir+"/Apple___Black_rot")

selected_images = os.listdir(resized_dir+"/Apple___Cedar_apple_rust")[:num_images_per_class]
for image in selected_images:
    shutil.copy(os.path.join(resized_dir+"/Apple___Cedar_apple_rust", image), balanced_dir+"/Apple___Cedar_apple_rust")

selected_images = os.listdir(resized_dir+"/Apple___healthy")[:num_images_per_class]
for image in selected_images:
    shutil.copy(os.path.join(resized_dir+"/Apple___healthy", image), balanced_dir+"/Apple___healthy")

selected_images = os.listdir(resized_dir+"/Not___apple")[:num_images_per_class]
for image in selected_images:
    shutil.copy(os.path.join(resized_dir+"/Not___apple", image), balanced_dir+"/Not___apple")

In [ ]:
# Preview Gambar dari Dataset yang Seimbang
print("Preview Gambar dari Dataset yang Seimbang:")

for class_name in apple_classes:
    print(f"\nKelas: {class_name}")

    # Direktori kelas
    class_dir = os.path.join(balanced_dir, class_name)

    # Daftar gambar dalam kelas
    class_images = os.listdir(class_dir)

    # Total gambar dalam kelas
    num_images = len(class_images)
    print(f"Total Gambar: {num_images}")

    # Menampilkan beberapa gambar dari kelas
    plt.figure(figsize=(10, 5))
    plt.suptitle(f"Kelas: {class_name} (Total: {num_images} Gambar)", fontsize=16)

    for i, image_file in enumerate(class_images[:5]):  # Menampilkan hingga 5 gambar
        image_path = os.path.join(class_dir, image_file)
        image = mpimg.imread(image_path)

        # Menambahkan subplot untuk setiap gambar
        plt.subplot(1, 5, i + 1)
        plt.axis('off')
        plt.imshow(image)

        # Menambahkan judul deskriptif ke setiap gambar
        plt.title(f"Gambar {i + 1}\n{image.shape[0]}x{image.shape[1]} px", fontsize=10)

    plt.tight_layout()
    plt.subplots_adjust(top=0.85)  # Menyesuaikan untuk judul utama
    plt.show()

# MODELLING


In [ ]:
img_path = os.path.join(class_dir, img_file)


# Load image from file path
img = image.load_img(img_path, target_size=(224, 224))

# Convert PIL Image to NumPy array
img_array = image.img_to_array(img)

# Append the NumPy array to the list
train_images.append(img_array)

# Membagi dataset menjadi data latih dan data uji
train_dir = balanced_dir
train_images = []
train_labels = []

# Mengambil gambar dan label dari direktori
for class_name in apple_classes:
    class_dir = os.path.join(train_dir, class_name)
    class_images = os.listdir(class_dir)

    for img_file in class_images:
        img_path = os.path.join(class_dir, img_file)
        img = image.load_img(img_path, target_size=(224, 224))  # Mengubah ukuran gambar
        img_array = image.img_to_array(img)  # Mengubah gambar menjadi array
        train_images.append(img_array)
        train_labels.append(class_name)

# Mengonversi daftar menjadi array NumPy
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Mengonversi label menjadi angka
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Membagi data menjadi set latih dan set validasi
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels_encoded, test_size=0.2, random_state=42)

# Normalisasi data
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0

# Membuat model menggunakan MobileNetV2
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Menambahkan lapisan klasifikasi di atas model dasar
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(apple_classes), activation='softmax')  # Jumlah kelas
])

# Mengompilasi model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Menyiapkan callback untuk menghentikan pelatihan lebih awal
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Melatih model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=20,
                    batch_size=32,
                    callbacks=[early_stopping])

# Menyimpan model
model.save('/content/apple_classification_model.h5')

# Menampilkan akurasi pelatihan dan validasi
plt.plot(history.history['accuracy'], label='Akurasi Latih')
plt.plot(history.history['val_accuracy'], label='Akurasi Validasi')
plt.title('Akurasi Model')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'load_img'